# Для запуска в colab

In [9]:
# fast версия для pymotphy2 не работает на python3.7!!!
# Использовать только python3.6 либо python3.8
!pip install bigartm pymorphy2[fast]

     |████████████████████████████████| 378kB 6.0MB/s 
  Created wheel for DAWG: filename=DAWG-0.8.0-cp36-cp36m-linux_x86_64.whl size=858777 sha256=b74a43554715cd775ea67caa2e0fd2e38edb33bb40763ce88ea878709d3b8eb0
  Stored in directory: /root/.cache/pip/wheels/3d/1f/f0/a5b1f9d02e193c997d252c33d215f24dfd7a448bc0166b2a12
Successfully built DAWG


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Библиотеки

In [46]:
import itertools

from tqdm.notebook import tqdm
from nltk.tokenize import RegexpTokenizer
# не очень много стоп слов, но для задания хватит
from nltk.corpus import stopwords
import pandas as pd
import numpy as np
import pymorphy2
import nltk


In [36]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Работа с данными

## Загрузка датасета

In [4]:
data = pd.read_csv('drive/MyDrive/DATASETS/MachineLearning/topicmodeling/lenta-ru-filtered.csv')

## Анализ

In [5]:
data

,text,tags,len,date
0,С 1 сентября на всей территории России вводитс...,Все,1654,31-08-1999
1,"По сведениям миссии ООН, передаваемым РИА ""Нов...",Все,1086,31-08-1999
2,15 представителей национал-большевистской парт...,Все,1219,31-08-1999
3,Намеченная на сегодняшний день церемония вступ...,Все,3094,31-08-1999
4,"На юге Киргизии, а именно в Баткенском и Чон-А...",Все,1354,31-08-1999
...,...,...,...,...
863280,Популярное место среди туристов в Мурманской о...,Россия,1231,11-09-2020
863281,Рейтинги от международного рейтингового агентс...,NaN,1425,11-09-2020
863282,Российские ученые нашли в Якутии новый подвид ...,События,1299,11-09-2020
863283,Для указания коронавируса как причины смерти ч...,Общество,2061,11-09-2020


In [6]:
data.describe()

,len
count,863285.000000
mean,1341.245505
std,572.066583
min,3.000000
25%,980.000000
50%,1253.000000
75%,1596.000000
max,55387.000000


Видно, что новостные заметки занимают порядка 1.2 кб текста, что является порядка 3/4 страницы А4 печатного текста. Данная длина текста приемлема для анализа, так как не является слишком короткими (как например твиты, порядка 0.2 кб).

Заметим, что в данных присутсвует поле tags (посути это можно рассматривать как топик). Данное поле будет интерестно в последующем анализе.

In [32]:
topics = [x.lower() for x in np.unique(np.array(data['tags'].values, dtype=str)).tolist() if x != 'nan']

In [33]:
len(topics), topics[:10]

(94,
 ['69-я параллель',
  'coцсети',
  'авто',
  'автобизнес',
  'аналитика рынка',
  'английский футбол',
  'белоруссия',
  'бизнес',
  'бокс и мма',
  'вещи'])

## Предобработка

Выполним простую предобработку текстов:
- lowercase;
- лемматизация при помощи pymorphy2;
- удаление стоп слов из словаря nltk.

In [14]:
# Честно взято с реализации ru_sentencetokenizer от iPavlov.
# Переписано в класс (к сожалению очень давно переписал и быстро сами пакет от 
#          iPavlov найти не смог,мне быстрее было взять свой пофикшеный код...)
import re
import logging
from typing import Set, Tuple, List
class RuSentenceTokenizer(object):
    def __init__(self):
        self.SENT_RE = re.compile(r'[^\.?!…]+[\.?!…]*["»“]*')

        self._LAST_WORD = re.compile(r'(?:\b|\d)([a-zа-я]+)\.$', re.IGNORECASE)
        self._FIRST_WORD = re.compile(r'^\W*(\w+)')
        self._ENDS_WITH_ONE_LETTER_LAT_AND_DOT = re.compile(r'(\d|\W|\b)([a-zA-Z])\.$')
        self._HAS_DOT_INSIDE = re.compile(r'[\w]+\.[\w]+\.$', re.IGNORECASE)
        self._INITIALS = re.compile(r'(\W|\b)([A-ZА-Я]{1})\.$')
        self._ONLY_RUS_CONSONANTS = re.compile(r'^[бвгджзйклмнпрстфхцчшщ]{1,4}$', re.IGNORECASE)
        self._STARTS_WITH_EMPTYNESS = re.compile(r'^\s+')
        self._ENDS_WITH_EMOTION = re.compile(r'[!?…]|\.{2,}\s?[)"«»,“]?$')
        self._STARTS_WITH_LOWER = re.compile(r'^\s*[–-—-("«]?\s*[a-zа-я]')
        self._STARTS_WITH_DIGIT = re.compile(r'^\s*\d')
        self._NUMERATION = re.compile(r'^\W*[IVXMCL\d]+\.$')
        self._PAIRED_SHORTENING_IN_THE_END = re.compile(r'\b(\w+)\. (\w+)\.\W*$')

        self._JOIN = 0
        self._MAYBE = 1
        self._SPLIT = 2

        self.JOINING_SHORTENINGS = {'mr', 'mrs', 'ms', 'dr', 'vs', 'англ', 'итал', 'греч', 'евр', 'араб', 'яп', 'слав', 'кит',
                               'тел', 'св', 'ул', 'устар', 'им', 'г', 'см', 'д', 'стр', 'корп', 'пл', 'пер', 'сокр', 'рис'}
        self.SHORTENINGS = {'co', 'corp', 'inc', 'авт', 'адм', 'барр', 'внутр', 'га', 'дифф', 'дол', 'долл', 'зав', 'зам', 'искл',
                       'коп', 'корп', 'куб', 'лат', 'мин', 'о', 'обл', 'обр', 'прим', 'проц', 'р', 'ред', 'руб', 'рус', 'русск',
                       'сан', 'сек', 'тыс', 'эт', 'яз', 'гос', 'мн', 'жен', 'муж', 'накл', 'повел', 'букв', 'шутл', 'ед'}

        self.PAIRED_SHORTENINGS = {('и', 'о'), ('т', 'е'), ('т', 'п'), ('у', 'е'), ('н', 'э')}
        
        # whitespaces
        self.start_whitespace = re.compile('^\s+')
        self.end_whitespace = re.compile('\s+$')
    
    
    def _regex_split_separators(self, text: str) -> [str]:
        return [x.strip() for x in self.SENT_RE.findall(text)]
    
    def _is_sentence_end(self, left: str, 
                               right: str,
                               shortenings: Set[str],
                               joining_shortenings: Set[str],
                               paired_shortenings: Set[Tuple[str, str]]) -> int:
        if not self._STARTS_WITH_EMPTYNESS.match(right):
            return self._JOIN

        if self._HAS_DOT_INSIDE.search(left):
            return self._JOIN

        left_last_word = self._LAST_WORD.search(left)
        lw = ' '
        if left_last_word:
            lw = left_last_word.group(1)

            if lw.lower() in joining_shortenings:
                return self._JOIN

            if self._ONLY_RUS_CONSONANTS.search(lw) and lw[-1].islower():
                return self._MAYBE

        pse = self._PAIRED_SHORTENING_IN_THE_END.search(left)
        if pse:
            s1, s2 = pse.groups()
            if (s1, s2) in paired_shortenings:
                return self._MAYBE

        right_first_word = self._FIRST_WORD.match(right)
        if right_first_word:
            rw = right_first_word.group(1)
            if (lw, rw) in paired_shortenings:
                return self._MAYBE

        if self._ENDS_WITH_EMOTION.search(left) and self._STARTS_WITH_LOWER.match(right):
            return self._JOIN

        initials = self._INITIALS.search(left)
        if initials:
            border, _ = initials.groups()
            if (border or ' ') not in "°'":
                return self._JOIN

        if lw.lower() in shortenings:
            return self._MAYBE

        last_letter = self._ENDS_WITH_ONE_LETTER_LAT_AND_DOT.search(left)
        if last_letter:
            border, _ = last_letter.groups()
            if (border or ' ') not in "°'":
                return self._MAYBE
        if self._NUMERATION.match(left):
            return self._JOIN
        return self._SPLIT
    
    def tokenize(self, text: str,
                       shortenings: Set[str] = None,
                       joining_shortenings: Set[str] = None,
                       paired_shortenings: Set[Tuple[str, str]] = None) -> List[str]:
        
        spans = self.span_tokenize(text, shortenings, joining_shortenings, paired_shortenings)
        
        sentences = []
        for span in spans:
            sentences.append(text[span[0]:span[1]])
        
        return sentences
    
    def _span_strip(self, text, sent_start, span_end):
        # delete whitespace start spans
        start_whitespace = self.start_whitespace.search(text[sent_start: span_end])
        if start_whitespace:
            sent_start += (start_whitespace.span(0)[1] - start_whitespace.span(0)[0])
        # delete whitespace end spans
        end_whitespace = self.end_whitespace.search(text[sent_start: span_end])
        if end_whitespace:
            span_end -= (end_whitespace.span(0)[1] - end_whitespace.span(0)[0])
            
        return sent_start, span_end
    
    def span_tokenize(self, text,
                            shortenings: Set[str] = None,
                            joining_shortenings: Set[str] = None,
                            paired_shortenings: Set[Tuple[str, str]] = None) -> List[Tuple]:
        
        if shortenings is None:
            shortenings = self.SHORTENINGS
        if joining_shortenings is None:
            joining_shortenings = self.JOINING_SHORTENINGS
        if paired_shortenings is None:
            paired_shortenings = self.PAIRED_SHORTENINGS
    
        spans = []
        sents = self._regex_split_separators(text)
        si = 0
        processed_index = 0
        sent_start = 0
        while si < len(sents):
            s = sents[si]
            span_start = text[processed_index:].index(s) + processed_index
            span_end = span_start + len(s)
            processed_index += len(s)

            si += 1

            send = self._is_sentence_end(text[sent_start: span_end], 
                                         text[span_end:],
                                         shortenings, 
                                         joining_shortenings, 
                                         paired_shortenings)
            if send == self._JOIN:
                continue

            if send == self._MAYBE:
                if self._STARTS_WITH_LOWER.match(text[span_end:]):
                    continue
                if self._STARTS_WITH_DIGIT.match(text[span_end:]):
                    continue
                    
            sent_start, span_end = self._span_strip(text, sent_start, span_end)

            if not text[sent_start: span_end].strip():
                logging.debug("Something went wrong while tokenizing")
            else:
                spans.append((sent_start, span_end))
            
            sent_start = span_end
            processed_index = span_end

        if sent_start != len(text):
            if text[sent_start:].strip():
                sent_start, span_end = self._span_strip(text, sent_start, len(text))
                spans.append((sent_start, span_end))
        return spans

In [37]:
# уже давно пользуюсь для себя унифицированными токенизаторами
word_tokenizer = RegexpTokenizer('[a-zа-яёЁА-ЯA-Z]+|[^\w\s]|\d+')
sent_tokenizer = RuSentenceTokenizer()
# pymorphy2 анализатор
morph = pymorphy2.MorphAnalyzer()
# стоп слова из nltk
stops = stopwords.words('russian')

In [41]:
def proccess_text(text):
    text= text.lower()
    sents = sent_tokenizer.tokenize(text)
    words = list(
        itertools.chain.from_iterable(
            word_tokenizer.tokenize_sents(sents)))
    return [x for x in [morph.normal_forms(word)[0] for word in words ]\
            if x not in stops]

In [49]:
for i in tqdm(range(len(data))):
    data.loc[i]['text'] = ' '.join(proccess_text(data.loc[i]['text']))

# препроцесинг занимает порядка двух часов, поэтому рекомендуется загрузисть
# сразу обработанную версию
# data = pd.read_csv('drive/MyDrive/DATASETS/MachineLearning/topicmodeling/lenta-ru-proccess.csv')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


KeyboardInterrupt: ignored